In [0]:
#Ishank Jain
#20757588
#libraries
from __future__ import print_function
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D       #CNN layers
from keras.datasets import mnist                     #Dataset
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from scipy.ndimage.filters import gaussian_filter
from PIL import Image
from PIL import ImageFilter
import matplotlib.pyplot as plt
from keras import regularizers     #to add l2 regularisation

Using TensorFlow backend.


In [0]:
%matplotlib inline

In [0]:
def resize(images):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((32, 32))
        img = np.reshape(img, (32, 32, 1))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

In [0]:
def resize2(images):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((32, 32))
        #img = np.reshape(img, (32, 32, 1))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

In [0]:
def blurred(images, rad):
    blur = []
    for img in images:
        img = Image.fromarray(img)
        # Use PIL or other library of the sort to rotate
        img = gaussian_filter(img, sigma=rad)        #img = np.array(img)
        img = np.reshape(img, (32, 32, 1))
        blur.append(img)
    return np.array(blur)

In [0]:
def rotates(images, deg):
    rotated = []
    for img in images:
        img = Image.fromarray(img)
        # Use PIL or other library of the sort to rotate
        img = img.rotate(deg)
        #img = np.array(img)
        img = np.reshape(img, (32, 32, 1))
        rotated.append(img)
    return np.array(rotated)

In [0]:
batch_size = 32
num_classes = 10
epochs = 3

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(xtrain, y_train), (xtest, y_test) = mnist.load_data()

#converting data to 32x32 and normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
xtest = xtest.astype('float32')

In [0]:
x_train = resize(x_train)
x_test = resize(x_test)

xtest = resize2(xtest)
#x_train = blurred(x_train, 0)
#normalize data
x_train /= 255
x_test /= 255
xtest /= 255

    # convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
#block 1
model.add(Conv2D(64,(3, 3),
                 activation='relu', 
                 kernel_initializer='random_normal',
                 bias_initializer='zeros',
                 padding='same',
                 ))
model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 2
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
   #block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   # 
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   #
model.add(MaxPooling2D(pool_size=(2, 2)))
    #FC layer and flattening data from 3D to 2D ; REGULARISTION CONSTANT IS 0.001
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
    #Softmax layer
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=1e-5),
              metrics=['accuracy'])

history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[1])
print('Test accuracy:', score[0])


x_train shape: (60000, 32, 32, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 211s 4ms/step - loss: 6.6932 - acc: 0.7240 - val_loss: 4.7375 - val_acc: 0.9363
Epoch 2/3
60000/60000 [==============================] - 208s 3ms/step - loss: 4.0290 - acc: 0.9441 - val_loss: 3.3904 - val_acc: 0.9647
Epoch 3/3
60000/60000 [==============================] - 208s 3ms/step - loss: 2.9431 - acc: 0.9642 - val_loss: 2.4965 - val_acc: 0.9712
Test loss: 0.9712
Test accuracy: 2.496458936691284


In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
#list of degree to rotate image
degrees=list(range(-45,45,10))
for deg in degrees: 
    x_test1 = rotates(xtest, deg)
    score = model.evaluate(x_test1, y_test, verbose=0)
    print("For rotation {} degree, Test loss:{} and Test accuracy:{}".format(deg,score[0], score[1]))
    

For rotation -45 degree, Test loss:5.018269160461426 and Test accuracy:0.3742
For rotation -35 degree, Test loss:3.933498551940918 and Test accuracy:0.5844
For rotation -25 degree, Test loss:3.0806473770141602 and Test accuracy:0.787
For rotation -15 degree, Test loss:2.6538274894714355 and Test accuracy:0.9179
For rotation -5 degree, Test loss:2.526772190093994 and Test accuracy:0.9615
For rotation 5 degree, Test loss:2.5264440174102782 and Test accuracy:0.9636
For rotation 15 degree, Test loss:2.5755210418701173 and Test accuracy:0.9464
For rotation 25 degree, Test loss:2.829081106567383 and Test accuracy:0.8665
For rotation 35 degree, Test loss:3.4145967575073244 and Test accuracy:0.7056


In [0]:
#list of radius to blur image
radius=list(range(0,7,1))
for rad in radius: 
    x_test2 = blurred(xtest, rad)
    score = model.evaluate(x_test2, y_test, verbose=0)
    print("For blur radius {} , Test loss:{} and Test accuracy:{}".format(rad,score[0], score[1]))

For blur radius 0 , Test loss:2.496458936691284 and Test accuracy:0.9712
For blur radius 1 , Test loss:2.5594983589172364 and Test accuracy:0.9562
For blur radius 2 , Test loss:2.912530768203735 and Test accuracy:0.8471
For blur radius 3 , Test loss:3.5176089027404784 and Test accuracy:0.689
For blur radius 4 , Test loss:4.0745017677307125 and Test accuracy:0.4738
For blur radius 5 , Test loss:4.45915618057251 and Test accuracy:0.2872
For blur radius 6 , Test loss:4.670076180267334 and Test accuracy:0.1287
